# Multi-Modal LLM using Azure OpenAI GPT-4o-mini model for image reasoning

In this notebook, we show how to use **Azure** OpenAI GPT4o-mini MultiModal LLM class/abstraction for image understanding/reasoning.

In [ ]:
import sys
import os
sys.path.insert(1, '../')
import init_creds as creds
 
AZURE_OPENAI_API_KEY = creds.get_api_key()
AZURE_OPENAI_ENDPOINT = creds.get_endpoint()
 
if not AZURE_OPENAI_API_KEY:
    raise ValueError("No AZURE_OPENAI_API_KEY set for Azure OpenAI API")
if not AZURE_OPENAI_ENDPOINT:
    raise ValueError("No AZURE_OPENAI_ENDPOINT set for Azure OpenAI API")

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = "2024-07-01-preview"
os.environ["AZURE_OPENAI_API_VERSION"] = os.environ["OPENAI_API_VERSION"]

## Initialize `AzureOpenAIMultiModal` and Load Images from URLs

Unlike normal `OpenAI`, you need to pass a `engine` argument in addition to `model`. The `engine` is the name of your model deployment.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment="gpt-4o-mini",
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

In [ ]:
import base64
import httpx

image_url = "https://www.visualcapitalist.com/wp-content/uploads/2023/10/US_Mortgage_Rate_Surge-Sept-11-1.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")


In [ ]:
from IPython.display import HTML

HTML(f'<img width=400 src="data:image/jpeg;base64,{image_data}"/>')

### Complete a prompt with an image

In [ ]:
# Describe the image provided
# Identify the key observations and share the analysis with respect to real estate brokers
# Identify the key observations and share the analysis with respect to government officials
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Describe the image provided"),
        (
            "user",
            [
                {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{image_data}"},
                }
            ],
        ),
    ]
)

In [ ]:
chain = prompt | llm

In [ ]:
response = chain.invoke({"image_data": image_data})
print(response.content)